# Tagueando perguntas do Stackoverflow

### Bibliotecas básicas

In [24]:
import pandas as pd
import numpy as np

### Bibliotecas de visualização

In [25]:
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

### Carregamento dos dados

In [26]:
perguntas = pd.read_csv('datasets/stackoverflow_perguntas.csv')
perguntas.head()

,Perguntas,Tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js
1,"Gostaria de fazer testes unitários no Node.js,...",node.js
2,Como inverter a ordem com que o jQuery itera u...,jquery
3,Eu tenho uma página onde pretendo utilizar um ...,html
4,Como exibir os dados retornados do FireStore e...,html angular


In [27]:
print(f'Linhas: {perguntas.shape[0]} | Colunas: {perguntas.shape[1]}')

Linhas: 5408 | Colunas: 2


### Informações dos dados

In [28]:
perguntas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5408 entries, 0 to 5407
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Perguntas  5408 non-null   object
 1   Tags       5408 non-null   object
dtypes: object(2)
memory usage: 84.6+ KB


### Removendo os espaços das Tags

In [29]:
perguntas['Tags'] = perguntas['Tags'].apply(lambda x : x.strip())

### Lista das tags

In [30]:
print(len(perguntas['Tags'].unique()))
perguntas['Tags'].unique()

17


array(['node.js', 'jquery', 'html', 'html angular', 'angular',
       'jquery html', 'angular node.js', 'jquery html angular',
       'html jquery', 'jquery angular', 'html node.js', 'jquery node.js',
       'node.js html', 'angular html', 'html angular  node.js',
       'jquery html node.js', 'html angular node.js'], dtype=object)

#### Pegando as tags únicas

In [31]:
label = list()
for tags in perguntas['Tags'].unique():
    for tag in tags.split():
        if tag not in label:
            label.append(tag)
print(label)

['node.js', 'jquery', 'html', 'angular']


### Transformando labels em colunas

In [32]:
def nova_coluna(lista_tags, dataframe, nome_tags):
    for tag in lista_tags:
        coluna = list()
        for linha_tag in dataframe[nome_tags]:
            if tag in linha_tag:
                coluna.append(1)
            else:
                coluna.append(0)
        dataframe[tag] = coluna
        
nova_coluna(label, perguntas, 'Tags')

In [33]:
perguntas.head(10)

,Perguntas,Tags,node.js,jquery,html,angular
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1
5,Vi esse vídeo (link abaixo) e gostaria de cons...,html,0,0,1,0
6,Eu consigo fazer uma requisição de upload de u...,angular,0,0,0,1
7,Está apresentando o seguinte erro: 'ionic' nã...,node.js,1,0,0,0
8,Tenho um formulário reativo e eu preciso mostr...,angular,0,0,0,1
9,Eu estou com esse problema e não faço ideia de...,node.js,1,0,0,0


### Pré-processamento

In [34]:
import string, re, nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from unicodedata import normalize

stop_words = set(stopwords.words('portuguese'))

blankSpacesRegex = re.compile(r'\s{2,}')
punctuationRegex = re.compile(r'[^0-9a-zA-Z +_]')

def cleaning(text):
    cleanedText = text.lower()
    cleanedText = removeHtml(cleanedText)
    cleanedText = replaceBlanks(cleanedText)
    cleanedText = removeSpecialCharacters(cleanedText)
    cleanedText = removeStopwords(cleanedText)
    cleanedText = cleanText(cleanedText)    
    return cleanedText

def removeHtml(text):
    soup = BeautifulSoup(text,'html5lib')
    text = soup.get_text(strip = True)    
    return text

def replaceBlanks(text):
    text = text.replace(r'\r',' ')
    text = text.replace(r'\n', ' ')
    text = text.replace(r'\s', ' ')
    return text

def removeSpecialCharacters(text):
    text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = punctuationRegex.sub(' ', text)    
    return text

def removeStopwords(text):
    return ' '.join(word for word in text.split() if word not in stop_words)

def cleanText(text):
    text = blankSpacesRegex.sub(' ', str(text))
    text = text.strip()    
    return text

In [35]:
perguntas['Perguntas'] = perguntas['Perguntas'].apply(cleaning)
perguntas['Perguntas'][10]

'retornando alguns parametros preciso desmembrar parametros salvando duas tabelas segue codigo inicial lamina controller js code lamina service js code laminaresource java code simulacaoservice java code'

### Divisão dos dados

In [36]:
lista_zip_tags = list(zip(perguntas[label[0]], perguntas[label[1]], perguntas[label[2]], perguntas[label[3]]))
perguntas['todas_tags'] = lista_zip_tags
perguntas.head()

,Perguntas,Tags,node.js,jquery,html,angular,todas_tags
0,possuo projeto node js porem preciso criar exe...,node.js,1,0,0,0,"(1, 0, 0, 0)"
1,gostaria fazer testes unitarios node js utiliz...,node.js,1,0,0,0,"(1, 0, 0, 0)"
2,inverter ordem jquery itera array elementos ne...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3,pagina onde pretendo utilizar conjunto webwork...,html,0,0,1,0,"(0, 0, 1, 0)"
4,exibir dados retornados firestore diretiva ang...,html angular,0,0,1,1,"(0, 0, 1, 1)"


In [37]:
from sklearn.model_selection import train_test_split

perguntas_treino, perguntas_teste, tags_treino, tags_teste = train_test_split(perguntas['Perguntas'], perguntas['todas_tags'], 
                                                                              random_state=123, test_size=0.2, shuffle=True)

In [38]:
tags_treino_array = np.asarray(list(tags_treino))
tags_teste_array = np.asarray(list(tags_teste))

print(tags_treino_array)

[[0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 ...
 [0 1 1 0]
 [0 0 1 0]
 [0 1 1 0]]


### Relevância binária

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, max_df=0.85, stop_words=stop_words)),
    ('clf', OneVsRestClassifier(LogisticRegression(solver = 'lbfgs')))
])

clf.fit(perguntas_treino, tags_treino_array)
print('Accuracy: ',clf.score(perguntas_teste, tags_teste_array))

Accuracy:  0.4343807763401109


In [48]:
len(perguntas['todas_tags'].unique())
print(f'% escolha aleatória entre as possibilidades: {1/13}')

% escolha aleatória entre as possibilidades: 0.07692307692307693


<p align='justify'>Observando a acurácia do modelo, percebe-se que este está acertando, aproximadamente, 6x mais que uma escolha aleatória entre as 13 possibilidades.</p>

### Hamming Loss

In [49]:
from sklearn.metrics import hamming_loss

pred = clf.predict(perguntas_teste)
hamming_loss_onevsrest = hamming_loss(tags_teste_array, pred)
print("Hamming Loss {0: .2f}".format(hamming_loss_onevsrest))

Hamming Loss  0.18


<p align='justify'>Isso indica que a cada 100 previsões, o modelo acertará 82% dos casos.</p>

### Relevância binária com Com skmultilearn

In [50]:
from skmultilearn.problem_transform import BinaryRelevance

br = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, max_df=0.85, stop_words=stop_words)),
    ('clf', BinaryRelevance(LogisticRegression(solver = 'lbfgs')))
])

br.fit(perguntas_treino, tags_treino_array)
print('Accuracy: ', br.score(perguntas_teste, tags_teste_array))

previsao_br = br.predict(perguntas_teste)
hamming = hamming_loss(tags_teste_array, previsao_br)
print("Hamming Loss {0: .2f}".format(hamming))

Accuracy:  0.4343807763401109
Hamming Loss  0.18


### Classificação em cadeia

In [51]:
from skmultilearn.problem_transform import ClassifierChain

chain = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, max_df=0.85, stop_words=stop_words)),
    ('clf', ClassifierChain(LogisticRegression(solver = 'lbfgs')))
])

chain.fit(perguntas_treino, tags_treino_array)
print('Accuracy: ', chain.score(perguntas_teste, tags_teste_array))

previsao_cadeia = chain.predict(perguntas_teste)
hamming = hamming_loss(tags_teste_array, previsao_cadeia)
print("Hamming Loss {0: .2f}".format(hamming))

Accuracy:  0.5138632162661737
Hamming Loss  0.20


### MLkNN

In [52]:
from skmultilearn.adapt import MLkNN

mlknn = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, max_df=0.85, stop_words=stop_words)),
    ('clf', MLkNN())
])

mlknn.fit(perguntas_treino, tags_treino_array)
print('Accuracy: ', mlknn.score(perguntas_teste, tags_teste_array))

previsao_mlknn = mlknn.predict(perguntas_teste)
hamming = hamming_loss(tags_teste_array, previsao_mlknn)
print("Hamming Loss {0: .2f}".format(hamming))

Accuracy:  0.34473197781885395
Hamming Loss  0.24


### Comparando os classificadores

In [54]:
resultados_classificacao = pd.DataFrame()
resultados_classificacao['perguntas'] = perguntas_teste.values
resultados_classificacao['tags real'] = list(tags_teste)
resultados_classificacao['BR'] = list(previsao_br.toarray())
resultados_classificacao['cadeia'] = list(previsao_cadeia.toarray())
resultados_classificacao['mlknn'] = list(previsao_mlknn.toarray())
resultados_classificacao.head()

,perguntas,tags real,BR,cadeia,mlknn
0,conflito code code alguem sabe alguma forma re...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]"
1,fazendo site obrigado usar html css javascript...,"(0, 0, 1, 0)","[0, 1, 1, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]"
2,recentemente fiz refactor codigo adotar padrao...,"(1, 0, 0, 0)","[1, 0, 0, 0]","[1.0, 0.0, 0.0, 0.0]","[1, 0, 0, 0]"
3,codigo code passo valores code utilizo codigo ...,"(0, 1, 1, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 1, 0]"
4,ola funcao evento code deveria ativar marcar c...,"(0, 1, 1, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 1, 0]"
